In [1]:
import jax.random as jr
import mediapy as media
from IPython.display import Video
from jax import vmap
from self_organising_systems.biomakerca import environments as evm
from self_organising_systems.biomakerca.agent_logic import BasicAgentLogic
from self_organising_systems.biomakerca.mutators import (
    BasicMutator, RandomlyAdaptiveMutator)

from biomaker_utils import perform_evaluation, perform_simulation, start_simulation
from configs.base_config import BaselineConfig


import self_organising_systems.biomakerca.env_logic  as env_logic
import overrides.env_logic_override as env_logic_override
env_logic = env_logic_override



In [2]:
base_config = BaselineConfig()

st_env, env_config = evm.get_env_and_config(base_config.ec_id, width_type=base_config.env_width_type)
env_config.soil_unbalance_limit = base_config.soil_unbalance_limit

agent_logic = BasicAgentLogic(env_config, minimal_net=base_config.agent_model == "minimal")

sd = 1e-2 if base_config.mutator_type == "basic" and base_config.agent_model == "basic" else 1e-3
mutator = (
    BasicMutator(sd=sd, change_perc=0.2)
    if base_config.mutator_type == "basic"
    else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2)
)

print("Current config:")
print("\n".join("%s: %s" % item for item in vars(env_config).items()))

ku, key = jr.split(base_config.key)
programs = vmap(agent_logic.initialize)(jr.split(ku, base_config.n_max_programs))
programs = vmap(mutator.initialize)(jr.split(ku, programs.shape[0]), programs)

env = st_env

BasicAgentLogic.dsm_num_params = 0
BasicAgentLogic.nsl_num_params = 176
BasicAgentLogic.denm_num_params = 80
BasicAgentLogic.excl_num_params = 41
BasicAgentLogic.repr_num_params = 2
BasicAgentLogic.num_params = 299
Current config:
agent_state_size: 2
etd: DefaultTypeDef: {materials_list: ['VOID', 'AIR', 'EARTH', 'IMMOVABLE', 'SUN', 'OUT_OF_BOUNDS'], types: {'VOID': 0, 'AIR': 1, 'EARTH': 2, 'IMMOVABLE': 3, 'SUN': 4, 'OUT_OF_BOUNDS': 5, 'AGENT_UNSPECIALIZED': 6, 'AGENT_ROOT': 7, 'AGENT_LEAF': 8, 'AGENT_FLOWER': 9}, specialization_idxs: {'AGENT_UNSPECIALIZED': 0, 'AGENT_ROOT': 1, 'AGENT_LEAF': 2, 'AGENT_FLOWER': 3}, agent_types: [6 7 8 9], intangible_mats: [0 1], gravity_mats: [2 6 7 8 9], structural_mats: [6 7 8 9], propagate_structure_mats: [2 3 6 7 8 9], agent_spawnable_mats: [0 1 2], structure_decay_mats: [-1 -1  1  0 -1 -1  5  5  5  5], aging_mats: [6 7 8 9], dissipation_rate_per_spec: [[0.5 0.5],  [1.  1. ],  [1.  1. ],  [1.2 1.2]]}
env_state_size: 6
struct_integrity_cap: 200
absorb

In [3]:
step = 0

# Spring

In [4]:
spring_agent_logic = agent_logic

frame = start_simulation(env, base_config, env_config)
with media.VideoWriter(
	base_config.out_file, shape=frame.shape[:2], fps=base_config.fps, crf=18
) as video:
	perform_simulation(
		env, programs, base_config, env_config, spring_agent_logic, mutator, key, video, frame, step
	)

Video(base_config.out_file)

Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>
Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>


  0%|          | 0/125 [00:00<?, ?it/s]


TypeError: step_env() got an unexpected keyword argument 'air_diffusion_rate'

# Summer

In [ ]:
summer_agent_logic = agent_logic

frame = start_simulation(env, base_config, env_config)
with media.VideoWriter(
	base_config.out_file, shape=frame.shape[:2], fps=base_config.fps, crf=18
) as video:
	perform_simulation(
		env, programs, base_config, env_config, summer_agent_logic, mutator, key, video, frame, step
	)

Video(base_config.out_file)

  0%|          | 0/125 [00:00<?, ?it/s]

100%|██████████| 125/125 [00:44<00:00,  2.80it/s]


# Autumn

In [ ]:
autumn_agent_logic = agent_logic
summer_agent_logic.config.max_lifetime = 10000


frame = start_simulation(env, base_config, env_config)
with media.VideoWriter(
	base_config.out_file, shape=frame.shape[:2], fps=base_config.fps, crf=18
) as video:
	perform_simulation(
		env, programs, base_config, env_config, autumn_agent_logic, mutator, key, video, frame, step
	)

Video(base_config.out_file)

  0%|          | 0/125 [00:00<?, ?it/s]

100%|██████████| 125/125 [01:30<00:00,  1.38it/s]


# Winter

In [ ]:
# winter_agent_logic = agent_logic
# summer_agent_logic.config.max_lifetime = 100

# frame = start_simulation(env, base_config, env_config)
# with media.VideoWriter(
# 	base_config.out_file, shape=frame.shape[:2], fps=base_config.fps, crf=18
# ) as video:
# 	perform_simulation(
# 		env, programs, base_config, env_config, winter_agent_logic, mutator, key, video, frame, step
# 	)

# Video(base_config.out_file)

In [ ]:
perform_evaluation(env, programs, st_env, env_config, agent_logic, mutator, base_config)

Extracted 1 programs.
Took 21.268988609313965 seconds
Total number of agents [15009] 15009.0 0.0
Extinction events [0] 0.0 0.0
